In [ ]:
#TASK 1 - Please download dataset or generate it using the attached script dataset_generator.py
import pandas as pd
from tqdm import tqdm
import csv
import random
import string
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

random.seed(1)

letters = string.ascii_lowercase
letters_upper = string.ascii_uppercase
for _i in range(0, 10):
    letters += letters

for _i in range(0, 10):
    letters += letters_upper


def random_string(stringLength=10):
    """Generate a random string of fixed length """
    return ''.join(random.sample(letters, stringLength))


print("Products between {} and {}".format(1, 100000))
product_ids = [x for x in range(1, 100000)]
dates = ['2020-07-01', '2020-07-02', '2020-07-03', '2020-07-04', '2020-07-05', '2020-07-06', '2020-07-07', '2020-07-08', '2020-07-09', '2020-07-10']
seller_ids = [x for x in range(1, 10)]


#   Generate products
products = [[0, "product_0", 22]]
for p in tqdm(product_ids):
    products.append([p, "product_{}".format(p), random.randint(1, 150)])
#   Save dataframe
df = pd.DataFrame(products)
df.columns = ["product_id", "product_name", "price"]
df.to_csv("products.csv", index=False)
del df
del products

#   Generate sellers
sellers = [[0, "seller_0", 100000]]
for s in tqdm(seller_ids):
    sellers.append([s, "seller_{}".format(s), random.randint(0, 100000)])
#   Save dataframe
df = pd.DataFrame(sellers)
df.columns = ["seller_id", "seller_name", "daily_target"]
df.to_csv("sellers.csv", index=False)

#   Generate sales
total_rows = 100000
prod_zero = int(total_rows * 0.95)
prod_others = total_rows - prod_zero + 1
df_array = [["order_id", "product_id", "seller_id", "date", "num_pieces_sold", "bill_raw_text"]]
with open('sales.csv', 'w', newline='') as f:
    csvwriter = csv.writer(f)
    csvwriter.writerows(df_array)

order_id = 0
for i in tqdm(range(0, 40)):
    df_array = []

    for i in range(0, prod_zero):
        order_id += 1
        df_array.append([order_id, 0, 0, random.choice(dates), random.randint(1, 100), random_string(500)])

    with open('sales.csv', 'a', newline='') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerows(df_array)

    df_array = []
    for i in range(0, prod_others):
        order_id += 1
        df_array.append(
            [order_id, random.choice(product_ids), random.choice(seller_ids), random.choice(dates),
             random.randint(1, 100), random_string(500)])

    with open('sales.csv', 'a', newline='') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerows(df_array)

print("Done")

In [ ]:
#-------------------------TASK 1----------------------------------------------
#importing SparkSession class
from pyspark.sql import SparkSession

In [ ]:
#assigning config to spark variable
spark=SparkSession \
.builder \
.appName("Python Spark SQL basic example") \
.config("spark.some.config.option","some-value") \
.getOrCreate()

In [ ]:
#creating sparkcontext
sc=spark.sparkContext

In [ ]:
#2.1. Please upload data from sellers.csv into sellers_rdd variable. Use the textFile method with map function: 
sellers_rdd=sc.textFile("/home/jovyan/work/sellers.csv").map(lambda x:x.split(","))

In [ ]:
sellers_rdd_header=["seller_id", "seller_name", "daily_target"]

In [ ]:
#2.3. Save the first element of sellers_rdd into sellers_rdd_header variable
sellers_rdd_header=sellers_rdd.top(1)

In [ ]:
#2.4. Filter the sellers_rdd excluding the sellers_rdd_header value, and reassign it to sellers_rdd. 
sellers_rdd=sellers_rdd.filter(lambda x:x not in sellers_rdd_header)

In [ ]:
#2.	Create a DataFrame variable sellers_df_test using toDF() function.
sellers_df_test=sellers_rdd.toDF(schema=sellers_rdd_header[0])

In [ ]:
sellers_df_test.show()

In [ ]:
#3.	Run the following code in order to change data types 
sellers_rdd = sellers_rdd.map(lambda x: (int(x[0]),x[1],int(x[2])))

In [ ]:
#4.	Recreate a DataFrame variable sellers_df using createDataFrame() function. For this task you need to create a schema. 
#Review the dataset in order to choose correct data types.

In [ ]:
#create a schema
from pyspark.sql.types import *
sellers_rdd_head=StructType([StructField('seller_id',IntegerType(),True),
StructField('seller_name',StringType(),True),
StructField('daily_target',FloatType(),True)])

In [ ]:
#Recreate a DataFrame variable sellers_df using createDataFrame() function.
sellers_df=spark.createDataFrame(sellers_rdd,schema=sellers_rdd_head)

In [108]:
#Review the dataset in order to choose correct data types.
sellers_df.printSchema()

root
 |-- seller_id: integer (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- daily_target: float (nullable = true)



In [107]:
sellers_df.count()

Py4JJavaError: An error occurred while calling o597.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 33.0 failed 1 times, most recent failure: Lost task 0.0 in stage 33.0 (TID 46) (c90e108b8cc1 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/sql/session.py", line 1459, in prepare
    ver

In [ ]:
#5.	Create a Pandas dataframe sellers_df_pd  from Spark dataframe.
sellers_df_Pandas=sellers_df.toPandas()

In [ ]:
#2.2. Take first 10 elements of sellers_rdd.
sellers_rdd.take(10)

In [ ]:
import pandas as pd

In [ ]:
sellers_rdd_header

In [ ]:
sellers_rdd.collect()

In [ ]:
#2.5.2. Take top 3 elements of sellers_rdd in ascending order.
sellers_rdd.sortByKey(True,1).take(3)

In [ ]:
#2.5.1. Take top 3 elements of sellers_rdd in descending order.
sellers_rdd.sortByKey(False,1).take(3)

In [ ]:
#2.6. Get a number of partitions of sellers_rdd.
sellers_rdd.getNumPartitions()

In [ ]:
#---------------------TASK 3--------------------------------------
#3.1. Please create the calendar_rdd variable containing years starting from 1970 to 2099 using parallelized collections. 
calendar_rdd=sc.parallelize(range(1970,2100))

In [ ]:
#3.2. Filter the calendar_rdd data containing only coming years (2022, 2023..) and save it to the calendar_coming_years_rdd variable
import datetime
calendar_coming_years_rdd=calendar_rdd.filter(lambda x:int(datetime.datetime.today().strftime('%Y'))<=x)

In [ ]:
calendar_coming_years_rdd.collect()

In [ ]:
#3.3. Filter the calendar_rdd data containing only leap years (1972…2000…2016, 2020, 2024, …) and save it to the calendar_leap_years_rdd variable.
calendar_leap_years_rdd=calendar_rdd.filter(lambda x:x%4==0)

In [ ]:
calendar_leap_years_rdd.collect()

In [ ]:
#3.4. Create calendar_not_leap_years_rdd using some of SET operator and calendar_rdd and calendar_leap_years_rdd. It should contain all not leap years. Sort all values in ascending order and collect them.
calendar_non_leap_years_rdd=calendar_rdd.subtract(calendar_leap_years_rdd).sortBy(lambda x:x)

In [ ]:
calendar_non_leap_years_rdd.take(100)

In [ ]:
#3.5. Group values of calendar_rdd by millennium (use the 2000 value for grouping).
calendar_rdd.groupBy(lambda x: str(x)[0:1]).mapValues(list).collect()

In [ ]:
#3.6. Imagine that mistakenly you deleted calendar_rdd using command “del calendar_rdd”. Please try to get the same data using UNION command and save it to the new_calendar_rdd.
new_calendar_rdd=calendar_non_leap_years_rdd.union(calendar_leap_years_rdd)

In [ ]:
#3.7. You want to make sure that the UNION command above didn’t produce any duplicated data. Print count of elements of new_calendar_rdd and then a count of distinct values.
#Code to verify the duplicates
l=[]
for i,j in new_calendar_rdd.countByValue().items():
    if j>1:
        a=j+1
        print('Duplicate exists for key-> '+str(i)+' number of values-> '+str(j))
    else:
        l.append('No duplicates')
print(set(l))

In [ ]:
#Print count of elements of new_calendar_rdd and then a count of distinct values.
new_calendar_rdd.count()
new_calendar_rdd.countByValue()

In [ ]:
#3.8. Imagine that the time machine was created, and you are transported 1000 years ahead. Apply a map function to each element of calendar_rdd and create a calendar_rdd_future.   
calendar_rdd_future=calendar_rdd.map(lambda x:x+1000)

In [ ]:
calendar_rdd_future.collect()

In [ ]:
#3.9. Print the statistic of calendar_rdd data (count, min, max..)
calendar_rdd.stats()